In [51]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point



In [52]:
areas_pop_geom = gpd.read_file('data/new_data/mow-areas-pop-geom-final.csv',GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
areas_pop_geom['space'] = [0. for _ in range(areas_pop_geom.shape[0])]
areas_pop_geom['living_space'] = [0. for _ in range(areas_pop_geom.shape[0])]
areas_pop_geom.head()

,field_1,district_code,area_name,population,geometry,space,living_space
0,0,zo,Внуково,24687,"POLYGON ((37.24580 55.60191, 37.24185 55.59705...",0.0,0.0
1,1,vo,Восточный,13503,"POLYGON ((37.87250 55.82207, 37.87380 55.82000...",0.0,0.0
2,2,zo,Можайский,137634,"POLYGON ((37.33076 55.68532, 37.32943 55.68579...",0.0,0.0
3,3,svo,Бибирево,158939,"POLYGON ((37.64407 55.89770, 37.63561 55.89368...",0.0,0.0
4,4,vo,Богородское,110049,"POLYGON ((37.68745 55.81677, 37.68782 55.81691...",0.0,0.0


In [53]:
areas_to_space = dict()

In [64]:
houses_curr_pd = pd.read_csv('data/mow-reestr-geom/mow_reestr_geom_' + str(1) + '.csv', sep=';')
houses_curr_pd['area_num'] = [-1 for _ in range(houses_curr_pd.shape[0])]
houses_curr = gpd.GeoDataFrame(houses_curr_pd, geometry=gpd.GeoSeries.from_xy(houses_curr_pd['lon'], houses_curr_pd['lat']), crs=4326)
cols = houses_curr.columns
print(cols)

Index(['Адрес ОЖФ', 'Идентификационный код адреса дома в ГИС ЖКХ',
       'Глобальный уникальный идентификатор дома по ФИАС', 'Код ОКТМО',
       'Способ управления',
       'ОГРН организации, осуществляющей управление домом',
       'КПП организации, осуществляющей управление домом',
       'Наименование организации, осуществляющей управление домом', 'Тип дома',
       'Состояние', 'Общая площадь дома', 'Жилая площадь в доме',
       'Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития',
       'Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития',
       'Тип общежития', 'Тип помещения (блока)', 'Дата сноса объекта',
       'Номер помещения (блока)', 'Номер комнаты', 'Кадастровый номер',
       'Глобальный уникальный идентификатор дома',
       'Глобальный уникальный идентификатор помещения',
       'Глобальный уникальный идентификатор комнаты', 'lon', 'lat', 'area_num',
       'ge

In [71]:
l1 = 0

pds_dict = dict()

def check(area_row, house_row):
    if area_row.geometry.contains(house_row.geometry):
        if isinstance(house_row['Общая площадь дома'], str):
            house_row['Общая площадь дома'] = house_row['Общая площадь дома'].replace(',', '.')
        if isinstance(house_row['Жилая площадь в доме'], str):
            house_row['Жилая площадь в доме'] = house_row['Жилая площадь в доме'].replace(',', '.')

        return True, {'space': float(house_row['Общая площадь дома']), 'living_space': float(house_row['Жилая площадь в доме'])}
    return False, {}



for area_index, area_row in areas_pop_geom.iterrows():
    area_name = area_row['area_name']
    pds_dict[area_name] = {'space': 0, 'living_space': 0}
    houses_in_area = pd.DataFrame(columns=cols)
    print(area_index, area_name)
    count = 0
    for i in range(1, 14):
        houses_curr_pd = pd.read_csv('data/mow-reestr-geom/mow_reestr_geom_' + str(i) + '.csv', sep=';')
        houses_curr_pd['area_num'] = [-1 for _ in range(houses_curr_pd.shape[0])]
        houses_curr = gpd.GeoDataFrame(houses_curr_pd, geometry=gpd.GeoSeries.from_xy(houses_curr_pd['lon'], houses_curr_pd['lat']), crs=4326)
        for house_index, house_row in houses_curr.iterrows():
            contains, res = check(area_row, house_row)
            if contains:
                if not pd.isna(res['space']):
                    pds_dict[area_name]['space'] += res['space']
                if not pd.isna(res['living_space']):
                    pds_dict[area_name]['living_space'] += res['living_space']
                house_row['area_num'] = area_index
                houses_in_area = houses_in_area.append(house_row)
                count += 1
    print(count)
    houses_in_area.to_csv(f"data/mow-reestr-areas/{area_name}.csv", index=False)


for area_index, area_row in areas_pop_geom.iterrows():
    area_row['space'] = pds_dict[area_row['area_name']]['space']
    area_row['living_space'] = pds_dict[area_row['area_name']]['living_space']

areas_pop_geom.to_csv("data/areas-pop-geom-space.csv", index=False)
print(pds_dict)



0 Внуково


/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_71103/1392985403.py:24: DtypeWarning: Columns (18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  houses_curr_pd = pd.read_csv('data/mow-reestr-geom/mow_reestr_geom_' + str(i) + '.csv', sep=';')


819
1 Восточный


/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_71103/1392985403.py:24: DtypeWarning: Columns (18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  houses_curr_pd = pd.read_csv('data/mow-reestr-geom/mow_reestr_geom_' + str(i) + '.csv', sep=';')


26
2 Можайский


...

808
96 Таганский


/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_71103/1392985403.py:24: DtypeWarning: Columns (18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  houses_curr_pd = pd.read_csv('data/mow-reestr-geom/mow_reestr_geom_' + str(i) + '.csv', sep=';')


446
97 Тверской


/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_71103/1392985403.py:24: DtypeWarning: Columns (18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  houses_curr_pd = pd.read_csv('data/mow-reestr-geom/mow_reestr_geom_' + str(i) + '.csv', sep=';')


2326
{'Внуково': {'space': 378944.9999999997, 'living_space': 169846.5}, 'Восточный': {'space': 79190.8, 'living_space': 74027.70000000001}, 'Можайский': {'space': 3525138.100000003, 'living_space': 2951192.0}, 'Бибирево': {'space': 2572168.1999999993, 'living_space': 2430648.3}, 'Богородское': {'space': 2338132.299999998, 'living_space': 2171861.019999998}, 'Братеево': {'space': 1844167.5, 'living_space': 1758697.07}, 'Коптево': {'space': 2020449.6000000017, 'living_space': 1848440.5700000003}, 'Крылатское': {'space': 2045009.200000001, 'living_space': 1764232.1999999997}, 'Некрасовка': {'space': 3066827.2999999984, 'living_space': 2284600.1099999994}, 'Проспект Вернадского': {'space': 2147793.6, 'living_space': 1661033.54}, 'Ростокино': {'space': 1018546.6999999996, 'living_space': 855105.4}, 'Свиблово': {'space': 1914865.9000000001, 'living_space': 1407730.6999999997}, 'Северный': {'space': 1529799.500000001, 'living_space': 1123194.6}, 'Сокольники': {'space': 1418882.4000000001, 'l